## Import Libraries

In [20]:
import os
import time
import copy


# 데이터 분석 라이브러리
import numpy as np
import pandas as pd

# 시각화 라이브러리
import matplotlib as mpl
import matplotlib.pyplot as plt

from PIL import Image

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import timm

from __future__ import print_function, division
from torch.optim import lr_scheduler
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from sklearn.metrics import f1_score

cudnn.benchmark = True
plt.ion()   # interactive mode

In [22]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
device

device(type='cuda', index=0)

## Bring csv file with path & label

In [23]:
traindata_path = pd.read_csv('train_data.csv')
valdata_path = pd.read_csv('val_data.csv')
testdata_path = pd.read_csv('test_data.csv')

In [24]:
print(len(traindata_path))

15126


## Hyper Parameter
Resize([224,224]) - https://www.programcreek.com/python/example/104834/torchvision.transforms.Resize

In [25]:
transform = transforms.Compose([
        transforms.Resize([224,224]),
#         transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
batch_size = 256
num_workers = 2

## Dataset

In [26]:
def readImage(path_data):
    path_to_image = []
    for path in path_data:
        img = Image.open(path)
        path_to_image.append(img)
    return path_to_image

In [27]:
class MyDataset(Dataset):
    def __init__(self, data, transform, train):
        self._repr_indent = 4
        self.data = data
        self.X = readImage(self.data['img_path'])
        self.y = self.data['label']
        self.transform = transform
        self.train = train
        self.classes = list(set(self.y))
    
    def __getitem__(self, idx):
        X, y = self.X[idx], None
#         X, y = self.X[idx], -1

        if self.transform:
            X = self.transform(X)
        if self.train:
            y = self.y[idx]

        return X,y
#         return torch.tensor(X), torch.tensor(y)
#         return X.clone().detach(), y.clone().detach()
    
    def __len__(self):
        return len(self.X)
    
    def __repr__(self):
        '''
        https://github.com/pytorch/vision/blob/master/torchvision/datasets/vision.py
        '''
        head = "My Custom Dataset : Mask Dataset"
        num_data = self._repr_indent*" " + "Number of datapoints: {}".format(self.__len__())
        num_classes = self._repr_indent*" " + "Number of classes: {}".format(len(self.classes))
        return '\n'.join([head, num_data, num_classes])

In [28]:
train_dataset = MyDataset(
    data = traindata_path,
    transform = transform,
    train = True
)
val_dataset = MyDataset(
    data = valdata_path,
    transform = transform,
    train = True
)
test_dataset = MyDataset(
    data = testdata_path,
    transform = transform,
    train = False
)

In [29]:
train_dataset

My Custom Dataset : Mask Dataset
    Number of datapoints: 15126
    Number of classes: 18

In [30]:
val_dataset

My Custom Dataset : Mask Dataset
    Number of datapoints: 3774
    Number of classes: 18

In [31]:
test_dataset

My Custom Dataset : Mask Dataset
    Number of datapoints: 12600
    Number of classes: 1

## DataLoader

In [32]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size = batch_size,
    num_workers = num_workers,
    shuffle = True,
    drop_last = False
)
val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size = batch_size,
    num_workers = num_workers,
    shuffle = True,
    drop_last = False
)
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size = batch_size,
    num_workers = num_workers,
    shuffle = True,
    drop_last = False
)

In [33]:
dataloaders = {
    'train': train_loader,
    'val': val_loader,
    'test': test_loader
}

In [34]:
dataloaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x7f35649e9520>,
 'val': <torch.utils.data.dataloader.DataLoader at 0x7f35649e9850>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7f35649e9940>}

## Model
model 생성 및 확인 - https://towardsdatascience.com/getting-started-with-pytorch-image-models-timm-a-practitioners-guide-4e77b4bf9055

In [35]:
# timm.list_models('*vit*',pretrained=True)

In [36]:
model = timm.create_model('vit_small_patch16_224', pretrained=True, num_classes=18)

# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, 18)

# model.get_classifier()

In [37]:
# model(torch.randn(64,3,224,224)).shape

In [38]:
model = model.to(device)

loss_fn = torch.nn.CrossEntropyLoss() 

# Observe that all parameters are being optimized
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001) 

In [39]:
# model.default_cfg

In [40]:
import torchsummary
# torchsummary.summary(model,(3,224,224))

## Train
transfer learning train model - https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html   
save checkpoint - https://tutorials.pytorch.kr/recipes/recipes/saving_and_loading_a_general_checkpoint.html   
f1 score - https://eunsukimme.github.io/ml/2019/10/21/Accuracy-Recall-Precision-F1-score/   

In [41]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [44]:
def train_model(model, loss_fn, optimizer, num_epochs=25):
    NUM_ACC = 2
    since = time.time()   
    best_acc = 0.0

    for epoch in range(num_epochs):
        # Each epoch has a training and validation phase
        for phase in ["train", "val"]:
            i=0
            running_loss = 0.
            running_acc = 0.
            if phase == "train":
                model.train() 
            elif phase == "val":
                model.eval() 
            optimizer.zero_grad() 

            for images, labels in tqdm(dataloaders[phase]):
                images = images.to(device)
                labels = labels.to(device)

                with torch.set_grad_enabled(phase == "train"):
                    logits = model(images)
                    _, preds = torch.max(logits, 1)
                    loss = loss_fn(logits, labels)

                    if phase == "train":
                        loss.backward() 
                        i+=1
                        if i % NUM_ACC ==0: 
                            optimizer.step() 
                            optimizer.zero_grad() 

                running_loss += loss.item() * images.size(0) 
                running_acc += torch.sum(preds == labels.data)


            epoch_loss = running_loss / len(dataloaders[phase].dataset) 
            epoch_acc = running_acc / len(dataloaders[phase].dataset)
            print('Epoch {}/{} - {} Loss: {:.4f} Acc: {:.4f} '.format(epoch, num_epochs-1, phase, epoch_loss, epoch_acc))
            
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    return model

In [45]:
model = train_model(model, loss_fn, optimizer, num_epochs=10)


Epoch 0/9 - train Loss: 1.1730 Acc: 0.6539 



Epoch 0/9 - val Loss: 0.5457 Acc: 0.8256 




Epoch 1/9 - train Loss: 1.4184 Acc: 0.5543 



Epoch 1/9 - val Loss: 1.5168 Acc: 0.5074 




Epoch 2/9 - train Loss: 1.0944 Acc: 0.6408 


...


Epoch 7/9 - val Loss: 2.3563 Acc: 0.4473 




Epoch 8/9 - train Loss: 0.0828 Acc: 0.9815 



Epoch 8/9 - val Loss: 2.5096 Acc: 0.4571 




Epoch 9/9 - train Loss: 0.0559 Acc: 0.9874 



Epoch 9/9 - val Loss: 2.5994 Acc: 0.4642 

Training complete in 10m 38s
Best val Acc: 0.825649
